In [1]:
import os 
%pwd

'/Users/somtoachi-kanu/End to End Folder/ML-project/notebooks'

In [2]:
os.chdir("../")
%pwd

'/Users/somtoachi-kanu/End to End Folder/ML-project'

In [3]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path
    preprocessor_path: Path

In [4]:
from marketing_campaign_responses.constants import *
from marketing_campaign_responses.utils.common import read_yaml, create_directories

In [5]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    
    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation

        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            preprocessor_path=config.preprocessor_path
        )

        return data_transformation_config


In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import RobustScaler, OrdinalEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE, RandomOverSampler
from collections import Counter
from marketing_campaign_responses import logger
import joblib

[2024-06-18 13:36:06,859: WARNING: font_manager: Matplotlib is building the font cache; this may take a moment.]
[2024-06-18 13:36:16,025: INFO: font_manager: Failed to extract font properties from /System/Library/Fonts/LastResort.otf: tuple indices must be integers or slices, not str]
[2024-06-18 13:36:16,160: INFO: font_manager: Failed to extract font properties from /System/Library/Fonts/Supplemental/NISC18030.ttf: In FT2Font: Could not set the fontsize (invalid pixel size; error code 0x17)]
[2024-06-18 13:36:16,163: INFO: font_manager: Failed to extract font properties from /System/Library/Fonts/Apple Color Emoji.ttc: In FT2Font: Could not set the fontsize (invalid pixel size; error code 0x17)]
[2024-06-18 13:36:16,180: INFO: font_manager: generated new fontManager]


In [7]:
class DataTransformation:
    def __init__(self, config):
        self.config = config
        self.preprocessor = None 
        self.transformed_df = None

    def get_data_transformation(self):
        try:
            # Load the dataset
            df = pd.read_csv(self.config.data_path)

            #df['contact'] = df['contact'].replace('cellular', 'telephone')
            #df['poutcome'] = df['poutcome'].replace('unknown', 'other')


            # Divide the dataset into independent and dependent features
            X = df.drop(columns=["responded"], axis=1)
            y = df["responded"]

            logger.info("Dividing the dataset into independent and dependent features completed")

            # Create an instance of LabelEncoder
            label_encoder = LabelEncoder()

            # Fit the label encoder to your categorical labels (y) and transform them
            y_encoded = label_encoder.fit_transform(y)

            logger.info("Encoding Target variable completed")

            numeric_features = X.select_dtypes(exclude="object").columns
            categorical_features = X.select_dtypes(include="object").columns

            # Oversample the minority class using RandomOverSampler
            oversampler = RandomOverSampler()
            X_resampled, y_resampled = oversampler.fit_resample(X, y_encoded)

            # Define the pipeline
            num_pipeline = Pipeline(
                steps=[
                    ("imputer", SimpleImputer(strategy="median")),
                    ("scaler", RobustScaler())
                ])

            cat_pipeline = Pipeline(
                steps=[
                    ("imputer", SimpleImputer(strategy="most_frequent")),
                    ("ordinalencoder", OrdinalEncoder()),
                ])

            # Define the Preprocessor
            preprocessor = ColumnTransformer(transformers=[
                ("OrdinalEncoder", OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1), categorical_features),
                ("RobustScaler", RobustScaler(), numeric_features)
            ], remainder="passthrough")

            self.preprocessor = preprocessor

            # Transform the whole data using the preprocessor
            X_transformed = preprocessor.fit_transform(X_resampled)

            # Get the updated column names after ordinal encoding
            column_names = numeric_features.to_list() + categorical_features.to_list()

            # Combine X_transformed and y back into one Dataframe
            self.transformed_df = pd.DataFrame(X_transformed, columns=column_names)
            self.transformed_df["responded"] = y_resampled

            logger.info("Data preprocessing completed")

        except Exception as e:
            raise e

    def save_preprocessor(self):
        if self.preprocessor is not None:
            joblib.dump(self.preprocessor, self.config.preprocessor_path)
            logger.info(f"Preprocessor saved to {self.config.preprocessor_path}")
        else:
            logger.warning("Preprocessor is not available. Please call get_data_transformation to create it.") 

    def train_test_split(self, test_size=0.2, random_state=None):
        if self.preprocessor is None:
            raise ValueError("Preprocessor is not available. Please call get_data_transformation.")

        # Split the data into train and test sets
        train, test = train_test_split(self.transformed_df, test_size=test_size, random_state=random_state)

        # Save the encoded train and test sets in the form of CSV files
        train.to_csv(os.path.join(self.config.root_dir, "train.csv"), index=False)
        test.to_csv(os.path.join(self.config.root_dir, "test.csv"), index=False)

        logger.info("Split the data into train and test sets.")
        logger.info(f"Shape of train data: {train.shape}")
        logger.info(f"Shape of test data: {test.shape}")

In [8]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.get_data_transformation()
    data_transformation.save_preprocessor()
    data_transformation.train_test_split()
except Exception as e:
    raise e

[2024-06-18 13:36:55,323: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-06-18 13:36:55,325: INFO: common: yaml file: params.yaml loaded successfully]
[2024-06-18 13:36:55,327: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-06-18 13:36:55,329: INFO: common: created directory at: artifacts]
[2024-06-18 13:36:55,330: INFO: common: created directory at: artifacts/data_transformation]
[2024-06-18 13:36:55,334: INFO: 27098535: Dividing the dataset into independent and dependent features completed]
[2024-06-18 13:36:55,335: INFO: 27098535: Encoding Target variable completed]
[2024-06-18 13:36:55,345: INFO: 27098535: Data preprocessing completed]
[2024-06-18 13:36:55,349: INFO: 27098535: Preprocessor saved to artifacts/data_transformation/preprocessor.joblib]
[2024-06-18 13:36:55,353: INFO: 27098535: Split the data into train and test sets.]
[2024-06-18 13:36:55,354: INFO: 27098535: Shape of train data: (44, 9)]
[2024-06-18 13:36:55,354: INFO: 27098535